<a href="https://colab.research.google.com/github/b12halflife/capstone_Project_ml/blob/ver2/model_trash_calsivikasi2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import numpy as np
import pandas as pd
import tensorflow as tf
import zipfile
from PIL import Image
from matplotlib import pyplot as plt
from tensorflow.python.util import compat
from tensorflow.core.protobuf import saved_model_pb2
from google.protobuf import text_format
import pprint
import json
import os
from tensorflow.keras.losses import CategoricalCrossentropy

train model

In [11]:
!gdown 1o8628IZ2pftqf900P6V6gjhxSm40HqEB

Downloading...
From: https://drive.google.com/uc?id=1o8628IZ2pftqf900P6V6gjhxSm40HqEB
To: /content/ssd_mobilenet_v2_taco_2018_03_29.engine
100% 26.1M/26.1M [00:00<00:00, 150MB/s]


Dataset

In [12]:
!gdown 1UobB5zH9-qSvmOLmA5DAHpWfKNx12F4Z

Downloading...
From: https://drive.google.com/uc?id=1UobB5zH9-qSvmOLmA5DAHpWfKNx12F4Z
To: /content/data.zip
100% 2.62G/2.62G [00:21<00:00, 122MB/s]


In [14]:
!unzip data.zip -d taco_dataset

Archive:  data.zip
replace taco_dataset/annotations.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: taco_dataset/annotations.json  
replace taco_dataset/batch_1/000000.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: taco_dataset/batch_1/000000.jpg  
replace taco_dataset/batch_1/000001.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: taco_dataset/batch_1/000001.jpg  
  inflating: taco_dataset/batch_1/000003.jpg  
  inflating: taco_dataset/batch_1/000004.jpg  
  inflating: taco_dataset/batch_1/000005.jpg  
  inflating: taco_dataset/batch_1/000006.jpg  
  inflating: taco_dataset/batch_1/000007.jpg  
  inflating: taco_dataset/batch_1/000008.jpg  
  inflating: taco_dataset/batch_1/000010.jpg  
  inflating: taco_dataset/batch_1/000011.jpg  
  inflating: taco_dataset/batch_1/000012.jpg  
  inflating: taco_dataset/batch_1/000013.jpg  
  inflating: taco_dataset/batch_1/000014.jpg  
  inflating: taco_dataset/batch_1/000015.jpg  
  inflating: taco_dataset/batch_1/000

In [15]:
DATA_DIR = '/content/taco_dataset'

In [16]:
def extract_subcategory(file_name):
    return file_name[:3]


In [17]:
import os
import shutil

# Define paths
data_directory = '/content/taco_dataset'
output_directory = '/content/split'

# Create output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Iterate over the entries in the dataset directory
for entry in os.scandir(data_directory):
    if entry.is_file():
        image_file = entry.name
        # Get the subcategory from the image file name or path
        subcategory = extract_subcategory(image_file)  # Replace with your own method to extract subcategory
        
        if subcategory:
            # Create subcategory directory if it doesn't exist
            subcategory_directory = os.path.join(output_directory, subcategory)
            os.makedirs(subcategory_directory, exist_ok=True)
            
            # Copy the image to the corresponding subcategory directory
            shutil.copy(os.path.join(data_directory, image_file), subcategory_directory)


In [18]:
import os
import random
import shutil

# Define paths
train_ratio = 0.8  # Ratio for training data

# Create output directories
train_directory = os.path.join(output_directory, 'train')
val_directory = os.path.join(output_directory, 'val')
os.makedirs(train_directory, exist_ok=True)
os.makedirs(val_directory, exist_ok=True)

# Iterate over the subcategory directories
for subcategory in os.listdir(data_directory):
    subcategory_directory = os.path.join(data_directory, subcategory)
    
    # Skip non-directory entries
    if not os.path.isdir(subcategory_directory):
        continue
    
    # Create subcategory directories in the output directories
    train_subcategory_directory = os.path.join(train_directory, subcategory)
    val_subcategory_directory = os.path.join(val_directory, subcategory)
    os.makedirs(train_subcategory_directory, exist_ok=True)
    os.makedirs(val_subcategory_directory, exist_ok=True)
    
    # Get the list of image files in the subcategory directory
    image_files = os.listdir(subcategory_directory)
    random.shuffle(image_files)  # Shuffle the image files randomly
    
    # Split the image files based on the train_ratio
    train_count = int(len(image_files) * train_ratio)
    train_files = image_files[:train_count]
    val_files = image_files[train_count:]
    
    # Copy the train files to the train subcategory directory
    for file in train_files:
        shutil.copy(os.path.join(subcategory_directory, file), train_subcategory_directory)
    
    # Copy the validation files to the validation subcategory directory
    for file in val_files:
        shutil.copy(os.path.join(subcategory_directory, file), val_subcategory_directory)


In [19]:
# grader-required-cell

# Import the inception model  
from tensorflow.keras.applications.inception_v3 import InceptionV3

# Create an instance of the inception model from the local pre-trained weights
local_weights_file = '/content/ssd_mobilenet_v2_taco_2018_03_29.engine'

In [22]:
def create_pre_trained_model(local_weights_file):
  
  pre_trained_model = InceptionV3(input_shape = (300, 300, 3),
                                  include_top = False, 
                                  weights = None) 

  pre_trained_model.load_weights(local_weights_file)

  for layer in pre_trained_model.layers:
        layer.trainable = False

  return pre_trained_model

In [23]:
pre_trained_model = create_pre_trained_model(local_weights_file)
pre_trained_model.summary()

OSError: ignored

In [24]:

# Build the model
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    #tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    #tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron.
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [25]:
from tensorflow.keras.optimizers import RMSprop

# Set training parameters
model.compile(loss='Categorical Cross-Entropy',
              optimizer=RMSprop(learning_rate=1e-4),
              metrics=['accuracy'])

In [29]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Apply data augmentation
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        '/content/split/train',  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 150x150
        batch_size=128,
        # Since we use Categorical Cross-Entropy loss, we need 'categorical' labels
        class_mode='categorical')

# Flow training images in batches of 128 using train_datagen generator
validation_generator = validation_datagen.flow_from_directory(
        '/content/split/val',  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 300x300
        batch_size=32,
        # Since we use Categorical Cross-Entropy loss, we need 'categorical' labels
        class_mode='categorical')

Found 1196 images belonging to 15 classes.
Found 304 images belonging to 15 classes.


In [35]:
# Constant for epochs
EPOCHS = 20

# Train the model
#history = model.fit(
#      train_generator,
#      steps_per_epoch=8,  
#      epochs=EPOCHS,
#      verbose=1,
#      validation_data = validation_generator,
#      validation_steps=8)
history = model.fit(train_images, train_labels, 
                    epochs=10, 
                    batch_size=32, 
                    validation_data=(test_images, test_labels))

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py:1176: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


InvalidArgumentError: ignored